# Model Pipeline

This notebook is used for training candidate models for air quality dataset.

In [1]:
MODEL_NAME = 'baseline'

## Modules

In [2]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

Using TensorFlow backend.


## Ensuring reproducibility

In [3]:
CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

## Data Preparation

### Load Dataset

In [4]:
path = '../dataset/Air quality/'
dataset = pd.read_csv(path + 'cleansed_air_quality.csv')
dataset = dataset.set_index('date')
dataset

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
date,,,,,,,,,,,,,
2013-03-01 00:00:00,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,WNW,3.1,Huairou
2013-03-01 00:00:00,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 00:00:00,8.0,8.0,6.0,28.0,400.0,52.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Wanliu
2013-03-01 00:00:00,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping
2013-03-01 00:00:00,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Guanyuan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-28 23:00:00,15.0,22.0,13.0,34.0,500.0,60.0,7.4,1014.9,-11.9,0.0,N,1.4,Shunyi
2017-02-28 23:00:00,20.0,25.0,6.0,28.0,900.0,54.0,7.0,1009.4,-12.2,0.0,N,1.9,Changping
2017-02-28 23:00:00,10.0,28.0,7.0,48.0,600.0,39.0,8.6,1014.1,-15.9,0.0,NNE,1.3,Nongzhanguan


### Splitting Dataset

In [5]:
# Split dataset
feat_cols = ["PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
excluded_cols = ["wd", "station"]
target = "PM2.5"
X, y = dataset[feat_cols], dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print ("Ukuran data training {}, data testing {}".format(X_train.shape, X_test.shape))

Ukuran data training (219323, 10), data testing (93996, 10)


## Load Models

Please copy your models here as a function.

In [6]:
# define base model
def baseline_model(input_dim):
    def _model():
        # create model
        model = Sequential()
        model.add(Dense(50, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    return _model

In [7]:
baseline_model(16)

<function __main__.baseline_model.<locals>._model()>

## Create Pipeline

### Model Definition

In [9]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('pca', PCA(n_components='mle', svd_solver='full')))
estimators.append(('normalizer', MinMaxScaler()))
estimators.append(('regressor', KerasRegressor(build_fn=baseline_model(X_train.shape[1]), epochs=50, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)

## Run Pipeline

In [10]:
pipeline.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.


ValueError: Error when checking input: expected dense_1_input to have shape (10,) but got array with shape (9,)

## Evaluation

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(r2)
print(rmse)

## Save Model

In [ ]:
from sklearn.externals import joblib
joblib.dump(pipeline.best_estimator_, MODEL_NAME + '.pkl')